In [ ]:
import pylab as py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import TransformerMixin,BaseEstimator
from datetime import datetime
from datetime import date
import scipy.stats as stats
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.impute import KNNImputer
pd.options.mode.chained_assignment = None  # default='warn'
sns.set_palette("mako")

In [ ]:
def find_outliers(data,column):
    data_mean, data_std=np.mean(data),np.std(data)
    cut_off=data_std*3
    lower, upper = data_mean - cut_off, data_mean + cut_off
    outliers=[x for x in data if x<lower or x >upper]
    print('Počet outlierov v stĺpci ' + column +  ' je %d' % len(outliers))

In [ ]:
def findDiffCorrelation(tmp_data):
    return (tmp_data.iloc[: , :15].corr() - data.iloc[: , :15].corr()).max()

In [ ]:
def transform_col(dataframe, column):
        quartil_down = dataframe[column].quantile(0.05)
        quartil_up = dataframe[column].quantile(0.95)
        
        for i, value in dataframe.iterrows():
            if value[column] < quartil_down:
                dataframe.loc[i, column] = quartil_down
            elif value[column] > quartil_up:
                dataframe.loc[i, column] = quartil_up
        return dataframe
def transform(dataframe):
    col_names= list(dataframe.columns.values)
    col_names = col_names[0:16]
    for column_name in col_names:
        transform_col(dataframe,column_name)
    return dataframe

### Načítanie a čistenie dát

In [ ]:
stations= pd.read_csv("data/stations.csv", sep="\t")
measurements= pd.read_csv("data/measurements.csv", sep="\t")

In [ ]:
measurements.info()

In [ ]:
stations.info()

Skontrolujeme, či sú jednotlivé 'latitude' jedinečné, a podľa nich joineme.

In [ ]:
latitude_s = stations["latitude"].unique().sort()
latitude_m = measurements["latitude"].unique().sort()
np.array_equal(latitude_s, latitude_m)

Sú rovnaké, preto môžeme ich spojiť.
Odstránime QoS, revision a code, keďže tieto parametre neovplyvňujú kvalitu ovzdušia.Taktiež odstránime duplicitný stĺpec ktorý nám vznikne.

In [ ]:
stations.drop(columns=["QoS"], inplace=True)
stations.drop(columns=["revision"], inplace=True)
stations.drop(columns=["code"], inplace=True)
data = pd.merge(measurements, stations, on = ["latitude"], how = "left")
data.drop(columns=["longitude_x"], inplace=True)
data.rename(columns={'longitude_y': 'longitude'}, inplace=True)
data_orig = data.copy()
data.info()

Ukážka dát po spojení.

In [ ]:
data.head()

Odstránime duplicitné záznamy

In [ ]:
tmp = data.duplicated()
tmp = tmp[tmp == True].count()
print('Počet duplicitných záznamov: '+ str(tmp))

In [ ]:
data = data.drop_duplicates()
tmp = data.duplicated()
tmp = tmp[tmp == True].count()
print('Počet duplicitných záznamov: '+ str(tmp))

In [ ]:
data = data.dropna()

In [ ]:
data.describe()

Presunieme si latitude a longtitude na koniec, aby sme vedeli neskôr ľahšie vyberať parametre ovplyvňujúce kvalitu ovzdušia.

In [ ]:
data.insert(16, 'latitude', data.pop('latitude'))
data.insert(17, 'longitude', data.pop('longitude'))
data.info()

### Odstránenie NA hodnôt
Ako prvé skúsime metódu s odstránením riadkov, kde chýbajú hodnoty.

In [ ]:
tmp_data = data.copy()
tmp_data.dropna(inplace=True)

In [ ]:
len(data) - len(tmp_data)

In [ ]:
plt.figure(figsize=(25,25))
sns.heatmap(data.iloc[: , :15].corr(), annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'BuGn');

In [ ]:
plt.figure(figsize=(25,25))
sns.heatmap(tmp_data.iloc[: , :15].corr(), annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'BuGn');

Výpočet korelácie metódou zmananím.

In [ ]:
findDiffCorrelation(tmp_data)

Výhodou tohto prístupu je, že sa nám heatmapa zmenila len minimálne. Nevýhodou je, že nám zmizlo 702 záznamov.
To nám predstavuje 5.87% dát.

In [ ]:
(len(data)-len(tmp_data))/(len(data)) * 100

2. metóda je nahradenie N/A hodnôt mediánom 

In [ ]:
temp_data_median = data.copy()
temp_data_median.fillna(temp_data_median.iloc[: , :15].median(), inplace=True)
print("Rozdiel v korelácii po nahradení chýbajúcich hodnôt mediánom")
findDiffCorrelation(temp_data_median)

Vykreslíme si heatmapu

In [ ]:
plt.figure(figsize=(25,25))
sns.heatmap(temp_data_median.iloc[: , :15].corr(), annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'BuGn');

3.Nahradenie chýbajúcich hodnôt priemerom


In [ ]:
tmp_data_average = data.copy()
tmp_data_average.fillna(tmp_data_average.iloc[: , :1].mean(), inplace=True)
print("Rozdiel v korelácii po nahradení chýbajúcich hodnôt priemerom")
findDiffCorrelation(tmp_data_average)

Vykreslíme si heatmapu

In [ ]:
plt.figure(figsize=(25,25))
sns.heatmap(tmp_data_average.iloc[: , :15].corr(), annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'BuGn');

In [ ]:
tmp_data_knn = data.copy()
imputer = KNNImputer(n_neighbors=2)
tmp_data_measurements = tmp_data_knn.iloc[:, :15]
tmp_data_measurements = imputer.fit_transform(tmp_data_measurements)
tmp_data_knn.iloc[:, :15] = tmp_data_measurements
print("Rozdiel v korelácii po nahradení chýbajúcich hodnôt kNN")
findDiffCorrelation(tmp_data_knn)


Vykreslíme si heatmapu

In [ ]:
plt.figure(figsize=(25,25))
sns.heatmap(tmp_data_knn.iloc[: , :15].corr(), annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'BuGn');

Nájdeme, ktorá z metód bola najefektívnejšia

In [ ]:
diff_corr_median = abs(temp_data_median.iloc[: , :15].corr() - data.iloc[: , :15].corr()).mean().mean()
diff_corr_average = abs(tmp_data_average.iloc[: , :15].corr() - data.iloc[: , :15].corr()).mean().mean()
diff_corr_knn = abs(tmp_data_knn.iloc[: , :15].corr() - data.iloc[: , :15].corr()).mean().mean()
print("Priemerná odchýlka korelácie od pôvodných dát po nahradení chýbajúcich hodnôt medianom: ", diff_corr_median)
print("Priemerná odchýlka korelácie od pôvodných dát  po nahradení chýbajúcich hodnôt priemerom: ", diff_corr_average)
print("Priemerná odchýlka korelácie od pôvodných dát  po nahradení chýbajúcich hodnôt kNN: ",diff_corr_knn)
listik = [diff_corr_median,diff_corr_average,diff_corr_knn]
print("Najmenšia odchýlka je:",min(listik))
  

Z priemerových odchýliek vidíme, že najlepšie pre nás bude použiť nahradenie hodnôt priemerom.   

In [ ]:
data = tmp_data_average.copy()

### Odstránenie vychýlených hodnôt (outlierov)

In [ ]:
for column in data.describe().columns:
    find_outliers(data[column],column)

Skopírujeme si dáta, aby sme vedeli vykonávať testy a nemenili sme priamo naše dáta.

In [ ]:
tmp_data = data.copy()

In [ ]:
transform(tmp_data)

In [ ]:
for column in data.describe().columns:
    find_outliers(tmp_data[column],column)

Ako vidíme, tak outlierov nám to úspešne odstránilo. Pri latitude a longitude outlierov neriešime - toto sú súradnice stanice.

In [44]:
tmp_data = data.copy()

UKRADNUTE OD SKURLIHO A NEFUNGUJE - KLASIKA

In [45]:
z_scores = stats.zscore(tmp_data.iloc[:,:16]) 
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
tmp_data = tmp_data[filtered_entries]
print(tmp_data)

           PAHs     PM10  C2H3NO5        O3      NH3        Pb        PRES  \
0       7.22628  9.07079  2.63533   9.95652  9.21864  29.98802  1093.16157   
3      10.39372  8.25541  1.19707   8.98772  9.82297  47.15361  1063.43896   
4       9.56441  8.43278  1.32137   4.43270  8.08426  47.94224  1111.55551   
7      10.78799  9.80691  5.28964   6.23420  8.38204  72.85236  1156.28181   
11      5.64192  9.36108  3.44736  12.02213  8.93778  45.68778  1185.27302   
...         ...      ...      ...       ...      ...       ...         ...   
25527  11.86566  7.54725  0.93197   8.97106  8.12568  74.87950  1123.93944   
25531   7.68875  9.53579  4.26800   9.41341  6.11207  25.79962  1194.52692   
25534   7.99479  9.64613  4.78692  10.14872  9.23772  66.96097  1167.87401   
25536   7.66997  9.33137  3.46029  10.38417  9.97797  41.85262  1134.35301   
25539   6.09774  6.50756  0.36799   7.84105  9.76842  23.67090  1169.72651   

           CFCs       SO2       NOx  warning       CO       CH4

In [46]:
tmp_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10711 entries, 0 to 25539
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PAHs       10711 non-null  float64
 1   PM10       10711 non-null  float64
 2   C2H3NO5    10711 non-null  float64
 3   O3         10711 non-null  float64
 4   NH3        10711 non-null  float64
 5   Pb         10711 non-null  float64
 6   PRES       10711 non-null  float64
 7   CFCs       10711 non-null  float64
 8   SO2        10711 non-null  float64
 9   NOx        10711 non-null  float64
 10  warning    10711 non-null  float64
 11  CO         10711 non-null  float64
 12  CH4        10711 non-null  float64
 13  PM2.5      10711 non-null  float64
 14  H2CO       10711 non-null  float64
 15  TEMP       10711 non-null  float64
 16  latitude   10711 non-null  float64
 17  longitude  10711 non-null  float64
 18  station    10711 non-null  object 
dtypes: float64(18), object(1)
memory usage: 1.6+ M

In [47]:
for column in data.describe().columns:
    find_outliers(tmp_data[column],column)

Počet outlierov v stĺpci PAHs je 1
Počet outlierov v stĺpci PM10 je 5
Počet outlierov v stĺpci C2H3NO5 je 244
Počet outlierov v stĺpci O3 je 4
Počet outlierov v stĺpci NH3 je 4
Počet outlierov v stĺpci Pb je 8
Počet outlierov v stĺpci PRES je 0
Počet outlierov v stĺpci CFCs je 7
Počet outlierov v stĺpci SO2 je 0
Počet outlierov v stĺpci NOx je 19
Počet outlierov v stĺpci warning je 0
Počet outlierov v stĺpci CO je 3
Počet outlierov v stĺpci CH4 je 2
Počet outlierov v stĺpci PM2.5 je 5
Počet outlierov v stĺpci H2CO je 8
Počet outlierov v stĺpci TEMP je 3
Počet outlierov v stĺpci latitude je 40
Počet outlierov v stĺpci longitude je 0


Ako vidíme, tak toto nám problém s outliermi nevyriešilo, použijeme teda nahradenie outlierov hraničnými hodnotami.

In [50]:
data = transform(data)

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11247 entries, 0 to 25539
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PAHs       11247 non-null  float64
 1   PM10       11247 non-null  float64
 2   C2H3NO5    11247 non-null  float64
 3   O3         11247 non-null  float64
 4   NH3        11247 non-null  float64
 5   Pb         11247 non-null  float64
 6   PRES       11247 non-null  float64
 7   CFCs       11247 non-null  float64
 8   SO2        11247 non-null  float64
 9   NOx        11247 non-null  float64
 10  warning    11247 non-null  float64
 11  CO         11247 non-null  float64
 12  CH4        11247 non-null  float64
 13  PM2.5      11247 non-null  float64
 14  H2CO       11247 non-null  float64
 15  TEMP       11247 non-null  float64
 16  latitude   11247 non-null  float64
 17  longitude  11247 non-null  float64
 18  station    11247 non-null  object 
dtypes: float64(18), object(1)
memory usage: 2.0+ M